In [1]:
%pip install docx2txt
%pip install fitz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade PyMuPDF

Note: you may need to restart the kernel to use updated packages.


In [3]:
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example
import pandas as pd
import json
from pathlib import Path
import random
import os
import docx2txt
import fitz  # PyMuPDF for PDF

In [4]:
print("spacy : ",spacy.__version__ )
print("pandas : ",pd.__version__ )
print("json : ",json.__version__ )
print("fitz : ",fitz.__version__ )

spacy :  3.7.2
pandas :  2.3.2
json :  2.0.9
fitz :  1.26.4


In [5]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [6]:
# 1. Load Excel

df = pd.read_excel(r"D:\CybaemTech\Projects\name_extraction_model\extract_text_dataset.xlsx")
df.head()

,Name,Content
0,ABHIJEET PATHAK,"ABHIJEET PATHAK Pune, India |+91-7694070046|ab..."
1,Ade Nikhil Nilkanth,Nikhil Nilkanth Latkar +91-8600395778 | E-mail...
2,Adnan Ahmad,Adnan Ahmad Email: adnanreboot@gmail.com | Pho...
3,Ajay A. Wagh,CURRICULUM VITAE Ajay A. Wagh Address : Plot n...
4,AJEET KUMAR PRAJAPATI,AJEET KUMAR PRAJAPATI C O N S U L T A N T http...


In [7]:
# Assume the column is called "data" (update with your actual column name)
TRAIN_DATA = []

In [8]:

for _, row in df.iterrows():
    try:
        data_dict = json.loads(row[0])  # assumes JSON is in first column
        name = str(data_dict["name"]).strip()
        text = str(data_dict["resume_text"]).strip()

        start = text.lower().find(name.lower())
        if start != -1:
            end = start + len(name)
            TRAIN_DATA.append((text, {"entities": [(start, end, "NAME")]}))
        else:
            print(f" Name '{name}' not found in resume text. Skipping.")
    except Exception as e:
        print(f" Error parsing row: {e}")

print(f" Training examples prepared: {len(TRAIN_DATA)}")

 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error parsing row: Expecting value: line 1 column 1 (char 0)
 Error p

C:\Users\abc\AppData\Local\Temp\ipykernel_6776\4226686023.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_dict = json.loads(row[0])  # assumes JSON is in first column


In [9]:
# Inspect the first column of the DataFrame
print(df.iloc[:, 0].dtype)
print(df.iloc[:, 0].head())
print(df.iloc[:, 0].isnull().sum())
print(df[df.iloc[:, 0] == ''].shape[0])

object
0          ABHIJEET PATHAK
1      Ade Nikhil Nilkanth
2              Adnan Ahmad
3             Ajay A. Wagh
4    AJEET KUMAR PRAJAPATI
Name: Name, dtype: object
0
0


In [10]:
# Inspect the 'Content' column of the DataFrame
print(df['Content'].dtype)
print(df['Content'].head())
print(df['Content'].isnull().sum())
print(df[df['Content'] == ''].shape[0])

object
0    ABHIJEET PATHAK Pune, India |+91-7694070046|ab...
1    Nikhil Nilkanth Latkar +91-8600395778 | E-mail...
2    Adnan Ahmad Email: adnanreboot@gmail.com | Pho...
3    CURRICULUM VITAE Ajay A. Wagh Address : Plot n...
4    AJEET KUMAR PRAJAPATI C O N S U L T A N T http...
Name: Content, dtype: object
0
0


In [11]:
TRAIN_DATA = []

for index, row in df.iterrows():
    try:
        name = str(row['Name']).strip()
        text = str(row['Content']).strip()

        # Case-insensitive search
        start = text.lower().find(name.lower())
        if start != -1:
            end = start + len(name)
            TRAIN_DATA.append((text, {"entities": [(start, end, "NAME")]}))
        else:
            print(f" Name '{name}' not found in resume text for row {index}. Skipping.")
    except Exception as e:
        print(f" Error processing row {index}: {e}")

print(f"Total training examples: {len(TRAIN_DATA)}")

 Name 'Ade Nikhil Nilkanth' not found in resume text for row 1. Skipping.
 Name 'Rahul Resume' not found in resume text for row 38. Skipping.
 Name 'Rutuja_s D.txt' not found in resume text for row 49. Skipping.
Total training examples: 60


In [12]:
# 2. Train SpaCy Model

nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")
ner.add_label("NAME")

optimizer = nlp.begin_training()

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    for itn in range(20):  # training iterations
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = spacy.util.minibatch(TRAIN_DATA, size=2)
        for batch in batches:
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], sgd=optimizer, losses=losses)
        print(f"Iteration {itn} Losses: {losses}")


d:\CybaemTech\Projects\name_extraction_model\name_extraction_model\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Bangalore,Karnataka India.PIN-560036EDAGOTTI HARIN..." with entities "[(36, 53, 'NAME')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\CybaemTech\Projects\name_extraction_model\name_extraction_model\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Configuring VPN and troubleshooting VPN issues (Gl..." with entities "[(1310, 1324, 'NAME')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Iteration 0 Losses: {'ner': 4008.8232962120355}
Iteration 1 Losses: {'ner': 45.25722351059748}
Iteration 2 Losses: {'ner': 265.78023728455145}
Iteration 3 Losses: {'ner': 29.015717362047983}
Iteration 4 Losses: {'ner': 3.9930283060287963}
Iteration 5 Losses: {'ner': 0.020256090898646394}
Iteration 6 Losses: {'ner': 0.0011082847561785351}
Iteration 7 Losses: {'ner': 1.7835934339853418e-07}
Iteration 8 Losses: {'ner': 1.6102974870326884e-07}
Iteration 9 Losses: {'ner': 1.514064158558074e-07}
Iteration 10 Losses: {'ner': 1.010971263653254e-06}
Iteration 11 Losses: {'ner': 1.7334639236464157e-05}
Iteration 12 Losses: {'ner': 6.80791028824181e-08}
Iteration 13 Losses: {'ner': 5.823842182343908e-07}
Iteration 14 Losses: {'ner': 7.760505054540291e-08}
Iteration 15 Losses: {'ner': 2.535499066273259e-07}
Iteration 16 Losses: {'ner': 4.825583891137131e-08}
Iteration 17 Losses: {'ner': 1.4212730606633098e-07}
Iteration 18 Losses: {'ner': 3.5198532287412597e-07}
Iteration 19 Losses: {'ner': 4.9344

In [23]:
# Save trained model
model_dir = Path("Name_extraction_model")
nlp.to_disk(model_dir)
print(f" Model saved to {model_dir.resolve()}")

 Model saved to D:\CybaemTech\Projects\name_extraction_model\name_extraction_model


In [24]:
# 3. Inference Functions

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text.strip()

def extract_text_from_docx(docx_path):
    text = docx2txt.process(docx_path)
    return text.strip()

def extract_name_from_resume(file_path, model_path="custom_name_ner"):
    nlp_model = spacy.load(model_path)

    ext = os.path.splitext(file_path)[-1].lower()
    if ext == ".pdf":
        resume_text = extract_text_from_pdf(file_path)
    elif ext == ".docx":
        resume_text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Only .pdf and .docx files are supported")

    doc = nlp_model(resume_text)
    names = [ent.text for ent in doc.ents if ent.label_ == "NAME"]

    return names[0] if names else None


In [25]:
# 4. Evaluate the Model

# Create a list of Example objects
examples = []
for text, annotations in TRAIN_DATA:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    examples.append(example)

# Evaluate the model
results = nlp.evaluate(examples) # Use the list of Example objects for evaluation
print("Evaluation Results:")
print(results)

# You can access specific metrics like:
print(f"  Precision: {results['token_p']}")
print(f"  Recall: {results['token_r']}")
print(f"  F-score: {results['token_f']}")

d:\CybaemTech\Projects\name_extraction_model\name_extraction_model\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Bangalore,Karnataka India.PIN-560036EDAGOTTI HARIN..." with entities "[(36, 53, 'NAME')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\CybaemTech\Projects\name_extraction_model\name_extraction_model\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Configuring VPN and troubleshooting VPN issues (Gl..." with entities "[(1310, 1324, 'NAME')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Evaluation Results:
{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 1.0, 'ents_r': 1.0, 'ents_f': 1.0, 'ents_per_type': {'NAME': {'p': 1.0, 'r': 1.0, 'f': 1.0}}, 'speed': 8225.450532980349}
  Precision: 1.0
  Recall: 1.0
  F-score: 1.0


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os

# Output widget for displaying results
output = widgets.Output()

upload_widget = widgets.FileUpload(
    accept='.pdf,.docx',  # Accepted file types
    multiple=False        # Single file upload
)
display(upload_widget)
display(output)

def handle_upload(change):
    output.clear_output()  # Clear previous output
    for file_info in upload_widget.value:
        content = file_info['content']
        name = file_info['name']
        ext = os.path.splitext(name)[-1].lower()
        temp_path = f'temp_uploaded{ext}'
        with open(temp_path, 'wb') as f:
            f.write(content)
        # Now use your extraction function
        extracted_name = extract_name_from_resume(temp_path, model_path="Name_extraction_model")
        with output:
            if extracted_name:
                print(f"Extracted Name: {extracted_name}")
            else:
                print("No name detected in the resume")

upload_widget.observe(handle_upload, names='value')

FileUpload(value=(), accept='.pdf,.docx', description='Upload')

Output()

In [18]:
import joblib
joblib.dump(nlp, "name_extraction_model.pkl")

['name_extraction_model.pkl']

In [19]:
print("model saved as name_extraction_model.pkl")

model saved as name_extraction_model.pkl
